<center><img src="../logo.png" alt="Header" style="width: 800px;"/></center>

# 颜色识别  Color Recognition

In [ ]:
# 中文开关，默认为英文 Chinese switch. The default value is English
g_ENABLE_CHINESE = False

Name_widgets = {
    'Close_Camera': ("Close_Camera", "关闭摄像头")
}

In [ ]:
# 显示摄像头组件 Camera display widgets
import ipywidgets.widgets as widgets
from IPython.display import display

import cv2
import time
import numpy as np


import threading

import HSV_Config

In [ ]:
#bgr8转jpeg格式
def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

In [ ]:
from SunriseRobotLib import Mipi_Camera

img_width=320
img_height=240

g_camera = Mipi_Camera(img_width, img_height)

if g_camera.isOpened():
    print("Open Camera OK")
else:
    print("Fail To Open Camera")


In [ ]:
update_hsv = HSV_Config.update_hsv()

Color_HSV  = {"Red"   : ((0, 70, 72), (7, 255, 255)),
              "Green" : ((54, 109, 78), (77, 255, 255)),
              "Blue"  : ((92, 100, 62), (121, 251, 255)),
              "Yellow": ((26, 100, 91), (32, 255, 255))}

g_stop_program = False

In [ ]:
# image_widget = widgets.Image(format='jpeg', width=320, height=240)
# 摄像头显示控件  Camera display widgets
DISPLAY_WIDTH = img_width
DISPLAY_HEIGHT = img_height
image_widget = widgets.Image(format='jpeg', width=DISPLAY_WIDTH, height=DISPLAY_HEIGHT)

In [ ]:
# 关闭摄像头 Close_Camera
Button_Close_Camera = widgets.Button(  
    value=False,  
    description=Name_widgets['Close_Camera'][g_ENABLE_CHINESE],      
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''     
    tooltip='Description',     
    icon='uncheck' )


# 按键按下事件处理   Key press event processing
def on_button_close_camera(b):
    global g_stop_program
    if b.description == Name_widgets['Close_Camera'][g_ENABLE_CHINESE]:
        # 停止线程，释放摄像头  Stop the thread and release the camera
        b.icon = 'uncheck'
        g_stop_program = True
        time.sleep(.1)
        g_camera.release()
    
# 关联按键事件回调 Button event callbacks
Button_Close_Camera.on_click(on_button_close_camera)

In [ ]:
# 图像处理任务 Image processing tasks
def task_processing():
    global g_stop_program
    t_start = time.time()
    m_fps = 0
    while g_camera.isOpened():
        if g_stop_program:
            break
        ret, frame = g_camera.read()
        frame, binary = update_hsv.get_contours(frame,Color_HSV)
        
        m_fps = m_fps + 1
        fps = m_fps / (time.time() - t_start)
        if (time.time() - t_start) >= 2:
            m_fps = fps
            t_start = time.time() - 1
        cv2.putText(frame, "FPS " + str(int(fps)), (10,20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,255), 1)
        
        # 图像传输给显示组件 The image is transmitted to the display component
        image_widget.value = bgr8_to_jpeg(frame)
        # time.sleep(.01)

In [ ]:
# 启动摄像头显示任务  Start the camera display task
thread1 = threading.Thread(target=task_processing)
thread1.setDaemon(True)
thread1.start()


# 创建一个盒子容器，以便将图像小部件相邻放置
# create a box container to place the image widget next to eachother
box_display = widgets.HBox([image_widget, Button_Close_Camera])
display(box_display)